In [6]:
import pandas as pd
import pymysql

# List of CSV file paths
csv_files = [
    "assam_bus_details.csv", "chandigarh_bus_details.csv", "himachal_bus_details.csv", 
    "kerala_bus_details.csv", "sb_bus_details.csv", "Telangana_bus_details.csv", 
    "up_bus_details.csv", "wb_bus_details.csv", "jk_bus_details.csv"
]

# Read each CSV file into a DataFrame and store it in a list
df_list = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames in the list
combined_df = pd.concat(df_list, ignore_index=True)

# Add ID column
combined_df.insert(0, "id", range(1, len(combined_df) + 1))

# Replace 'INR ' with an empty string in the 'Price' column
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '', regex=False)

# Extract the digits (0-9) from the 'Seat_Availability' column using a raw string for regex
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)')

# Drop rows with any NaN values
combined_df = combined_df.dropna()

# Connect to MySQL database
myconnection = pymysql.connect(host='localhost', user='root', passwd='1234', database="redbus")

# Prepare the SQL query to create a table with appropriate column types
a = ",".join(
    f"{i} {j}"
    for i, j in zip(combined_df.columns, combined_df.dtypes)
).replace("float64", "FLOAT").replace("object", "TEXT").replace("int64", "INT")

table_name = "bus_routes"
cursor = myconnection.cursor()

# Create the table if it doesn't exist
cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({a})")
myconnection.commit()

# Insert the data into the table
for i in range(len(combined_df)):
    cursor.execute(f"INSERT INTO {table_name} VALUES ({', '.join(['%s'] * len(combined_df.columns))})", tuple(combined_df.iloc[i]))
    myconnection.commit()

# Close the database connection
cursor.close()
myconnection.close()

# Optionally, save the combined DataFrame to a CSV file
combined_df.to_csv("bus_routes.csv", index=False)






